## Interaktívny Phi 3 Mini 4K Instruct Chatbot s Whisper

### Úvod:
Interaktívny Phi 3 Mini 4K Instruct Chatbot je nástroj, ktorý umožňuje používateľom komunikovať s Microsoft Phi 3 Mini 4K instruct demo pomocou textového alebo zvukového vstupu. Chatbot je možné použiť na rôzne úlohy, ako sú preklady, aktualizácie počasia a získavanie všeobecných informácií.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Vytvorte si svoj Huggingface prístupový token](https://huggingface.co/settings/tokens)

Vytvorte nový token  
Zadajte nový názov  
Vyberte oprávnenia na zápis  
Skopírujte token a uložte ho na bezpečné miesto


Nasledujúci Python kód vykonáva dve hlavné úlohy: importovanie modulu `os` a nastavenie environmentálnej premennej.

1. Importovanie modulu `os`:
   - Modul `os` v Pythone poskytuje spôsob, ako komunikovať s operačným systémom. Umožňuje vykonávať rôzne úlohy súvisiace s operačným systémom, ako napríklad prístup k environmentálnym premenným, práca so súbormi a adresármi, atď.
   - V tomto kóde je modul `os` importovaný pomocou príkazu `import`. Tento príkaz sprístupňuje funkcionalitu modulu `os` na použitie v aktuálnom Python skripte.

2. Nastavenie environmentálnej premennej:
   - Environmentálna premenná je hodnota, ku ktorej majú prístup programy bežiace na operačnom systéme. Je to spôsob, ako uchovávať konfiguračné nastavenia alebo iné informácie, ktoré môžu byť použité viacerými programami.
   - V tomto kóde sa nastavuje nová environmentálna premenná pomocou slovníka `os.environ`. Kľúč slovníka je `'HF_TOKEN'` a hodnota je priradená z premennej `HUGGINGFACE_TOKEN`.
   - Premenná `HUGGINGFACE_TOKEN` je definovaná tesne nad týmto úryvkom kódu a je jej priradená reťazcová hodnota `"hf_**************"` pomocou syntaxe `#@param`. Táto syntax sa často používa v Jupyter notebookoch na umožnenie vstupu používateľa a konfigurácie parametrov priamo v rozhraní notebooku.
   - Nastavením environmentálnej premennej `'HF_TOKEN'` je možné k nej pristupovať z iných častí programu alebo z iných programov bežiacich na rovnakom operačnom systéme.

Celkovo tento kód importuje modul `os` a nastavuje environmentálnu premennú s názvom `'HF_TOKEN'` s hodnotou, ktorá je poskytnutá v premennej `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Tento úryvok kódu definuje funkciu s názvom clear_output, ktorá slúži na vymazanie výstupu aktuálnej bunky v Jupyter Notebooku alebo IPython. Poďme si rozobrať kód a pochopiť jeho funkčnosť:

Funkcia clear_output prijíma jeden parameter s názvom wait, ktorý je logická hodnota (boolean). Predvolene je wait nastavený na hodnotu False. Tento parameter určuje, či by funkcia mala počkať, kým bude k dispozícii nový výstup na nahradenie existujúceho výstupu, predtým než ho vymaže.

Samotná funkcia sa používa na vymazanie výstupu aktuálnej bunky. V Jupyter Notebooku alebo IPython, keď bunka generuje výstup, ako napríklad vytlačený text alebo grafické vizualizácie, tento výstup sa zobrazí pod bunkou. Funkcia clear_output umožňuje tento výstup vymazať.

Implementácia funkcie nie je poskytnutá v úryvku kódu, čo je naznačené trojbodkou (...). Trojbodka predstavuje zástupný symbol pre skutočný kód, ktorý vykonáva vymazanie výstupu. Implementácia funkcie môže zahŕňať interakciu s API Jupyter Notebooku alebo IPython na odstránenie existujúceho výstupu z bunky.

Celkovo táto funkcia poskytuje pohodlný spôsob, ako vymazať výstup aktuálnej bunky v Jupyter Notebooku alebo IPython, čo uľahčuje správu a aktualizáciu zobrazovaného výstupu počas interaktívnych programovacích relácií.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Vykonajte text-to-speech (TTS) pomocou služby Edge TTS. Prejdime si relevantné implementácie funkcií jednu po druhej:

1. `calculate_rate_string(input_value)`: Táto funkcia prijíma vstupnú hodnotu a vypočíta reťazec rýchlosti pre hlas TTS. Vstupná hodnota predstavuje požadovanú rýchlosť reči, kde hodnota 1 predstavuje normálnu rýchlosť. Funkcia vypočíta reťazec rýchlosti odčítaním 1 od vstupnej hodnoty, vynásobením výsledku číslom 100 a následným určením znamienka na základe toho, či je vstupná hodnota väčšia alebo rovná 1. Funkcia vráti reťazec rýchlosti vo formáte "{sign}{rate}".

2. `make_chunks(input_text, language)`: Táto funkcia prijíma vstupný text a jazyk ako parametre. Rozdelí vstupný text na časti na základe jazykovo špecifických pravidiel. V tejto implementácii, ak je jazyk "English", funkcia rozdelí text pri každej bodke (".") a odstráni akékoľvek medzery na začiatku alebo na konci. Potom pridá bodku ku každej časti a vráti filtrovaný zoznam častí.

3. `tts_file_name(text)`: Táto funkcia generuje názov súboru pre audio súbor TTS na základe vstupného textu. Vykonáva niekoľko transformácií na texte: odstránenie bodky na konci (ak je prítomná), konverzia textu na malé písmená, odstránenie medzier na začiatku a na konci a nahradenie medzier podčiarkovníkmi. Potom text skráti na maximálne 25 znakov (ak je dlhší) alebo použije celý text, ak je prázdny. Nakoniec vygeneruje náhodný reťazec pomocou modulu [`uuid`] a skombinuje ho so skráteným textom na vytvorenie názvu súboru vo formáte "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Táto funkcia zlúči viacero audio súborov do jedného audio súboru. Prijíma zoznam ciest k audio súborom a výstupnú cestu ako parametre. Funkcia inicializuje prázdny objekt `AudioSegment` nazvaný [`merged_audio`]. Potom iteruje cez každú cestu k audio súboru, načíta audio súbor pomocou metódy `AudioSegment.from_file()` z knižnice `pydub` a pridá aktuálny audio súbor k objektu [`merged_audio`]. Nakoniec exportuje zlúčené audio do špecifikovanej výstupnej cesty vo formáte MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Táto funkcia vykonáva operáciu TTS pomocou služby Edge TTS. Prijíma zoznam textových častí, rýchlosť reči, názov hlasu a cestu na uloženie ako parametre. Ak je počet častí väčší ako 1, funkcia vytvorí adresár na uloženie jednotlivých audio súborov častí. Potom iteruje cez každú časť, zostaví príkaz Edge TTS pomocou funkcie `calculate_rate_string()`, názvu hlasu a textu časti a vykoná príkaz pomocou funkcie `os.system()`. Ak je vykonanie príkazu úspešné, pridá cestu k vygenerovanému audio súboru do zoznamu. Po spracovaní všetkých častí zlúči jednotlivé audio súbory pomocou funkcie `merge_audio_files()` a uloží zlúčené audio do špecifikovanej cesty na uloženie. Ak je len jedna časť, priamo vygeneruje príkaz Edge TTS a uloží audio do cesty na uloženie. Nakoniec vráti cestu na uloženie vygenerovaného audio súboru.

6. `random_audio_name_generate()`: Táto funkcia generuje náhodný názov audio súboru pomocou modulu [`uuid`]. Vygeneruje náhodné UUID, konvertuje ho na reťazec, vezme prvých 8 znakov, pridá príponu ".mp3" a vráti náhodný názov audio súboru.

7. `talk(input_text)`: Táto funkcia je hlavný vstupný bod na vykonanie operácie TTS. Prijíma vstupný text ako parameter. Najskôr skontroluje dĺžku vstupného textu, aby určila, či ide o dlhú vetu (väčšiu alebo rovnú 600 znakov). Na základe dĺžky a hodnoty premennej `translate_text_flag` určí jazyk a vygeneruje zoznam textových častí pomocou funkcie `make_chunks()`. Potom vygeneruje cestu na uloženie audio súboru pomocou funkcie `random_audio_name_generate()`. Nakoniec zavolá funkciu `edge_free_tts()` na vykonanie operácie TTS a vráti cestu na uloženie vygenerovaného audio súboru.

Celkovo tieto funkcie spolupracujú na rozdelení vstupného textu na časti, generovaní názvu súboru pre audio súbor, vykonaní operácie TTS pomocou služby Edge TTS a zlúčení jednotlivých audio súborov do jedného audio súboru.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementácia dvoch funkcií: convert_to_text a run_text_prompt, ako aj deklarácia dvoch tried: str a Audio.

Funkcia convert_to_text prijíma ako vstup audio_path a prepisuje zvuk na text pomocou modelu nazvaného whisper_model. Funkcia najprv kontroluje, či je nastavená hodnota gpu na True. Ak áno, whisper_model sa používa s určitými parametrami, ako sú word_timestamps=True, fp16=True, language='English' a task='translate'. Ak je gpu nastavené na False, whisper_model sa používa s fp16=False. Výsledný prepis sa potom uloží do súboru s názvom 'scan.txt' a vráti sa ako text.

Funkcia run_text_prompt prijíma ako vstup message a chat_history. Používa funkciu phi_demo na generovanie odpovede od chatbotu na základe vstupnej správy. Generovaná odpoveď sa potom odovzdá funkcii talk, ktorá odpoveď konvertuje na zvukový súbor a vráti cestu k súboru. Trieda Audio sa používa na zobrazenie a prehrávanie zvukového súboru. Zvuk sa zobrazuje pomocou funkcie display z modulu IPython.display a objekt Audio sa vytvára s parametrom autoplay=True, takže zvuk sa začne prehrávať automaticky. Chat_history sa aktualizuje o vstupnú správu a generovanú odpoveď, a vráti sa prázdny reťazec spolu s aktualizovaným chat_history.

Trieda str je vstavaná trieda v Pythone, ktorá predstavuje sekvenciu znakov. Poskytuje rôzne metódy na manipuláciu a prácu s reťazcami, ako sú capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill a ďalšie. Tieto metódy umožňujú vykonávať operácie ako vyhľadávanie, nahrádzanie, formátovanie a manipuláciu s reťazcami.

Trieda Audio je vlastná trieda, ktorá predstavuje zvukový objekt. Používa sa na vytvorenie prehrávača zvuku v prostredí Jupyter Notebook. Trieda prijíma rôzne parametre, ako sú data, filename, url, embed, rate, autoplay a normalize. Parameter data môže byť numpy pole, zoznam vzoriek, reťazec predstavujúci názov súboru alebo URL, alebo surové PCM dáta. Parameter filename sa používa na špecifikáciu lokálneho súboru, z ktorého sa načítajú zvukové dáta, a parameter url sa používa na špecifikáciu URL, z ktorého sa stiahnu zvukové dáta. Parameter embed určuje, či by zvukové dáta mali byť vložené pomocou data URI alebo odkazované z pôvodného zdroja. Parameter rate špecifikuje vzorkovaciu frekvenciu zvukových dát. Parameter autoplay určuje, či by sa zvuk mal začať prehrávať automaticky. Parameter normalize špecifikuje, či by zvukové dáta mali byť normalizované (škálované) na maximálny možný rozsah. Trieda Audio tiež poskytuje metódy ako reload na opätovné načítanie zvukových dát zo súboru alebo URL, a atribúty ako src_attr, autoplay_attr a element_id_attr na získanie príslušných atribútov pre zvukový prvok v HTML.

Celkovo sa tieto funkcie a triedy používajú na prepis zvuku na text, generovanie zvukových odpovedí od chatbotu a zobrazovanie a prehrávanie zvuku v prostredí Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
